#데이터과학특론
2020-2학기 구정현

In [1]:
!git clone https://github.com/picaku9/2020_2_data_final.git

Cloning into '2020_2_data_final'...
fatal: could not read Username for 'https://github.com': No such device or address


In [2]:
!git clone https://github.com/sdsatumd/fail.git

Cloning into 'fail'...
remote: Enumerating objects: 31, done.
remote: Total 31 (delta 0), reused 0 (delta 0), pack-reused 31
Unpacking objects: 100% (31/31), done.


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans

  Cloning https://github.com/tensorflow/cleverhans.git to /tmp/pip-install-nj35h2qo/cleverhans
  Running command git clone -q https://github.com/tensorflow/cleverhans.git /tmp/pip-install-nj35h2qo/cleverhans
     |████████████████████████████████| 163kB 4.0MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
  Created wheel for cleverhans: filename=cleverhans-3.0.1-cp36-none-any.whl size=262570 sha256=b9c85942c98c38560fc4ffa884116d5727bc4988e86e5d76c41c6a19fad9ad01
  Stored in directory: /tmp/pip-ephem-wheel-cache-6lvr5cb1/wheels/6e/59/ec/723a6f654aaf62c8c40f0f0850fdf71a4948598697f56c3bfa
Successfully built cleverhans


#돌아가는 코드

##cleverhans
###cleverhans_tutorial
1. mnist_tutorial_jsma.py

2. mnist_tutorial_blackbox.py

<li>Test accuracy of oracle on adversarial examples generated using the substitute: 0.565989847715736


#FAIL - 수정사항
###jsma.script.py


dataset.py -> from sklearn.model_selection import StratifiedShuffleSplit

Drebin.py ->  scores = model_selection.cross_val_score(clf, features, labels, cv=5)
from sklearn import svm, model_selection

spsa.py -> delete addons

ModelBasicCNN
jsma_script.py -> from cleverhans.model_zoo.basic_cnn import ModelBasicCNN

import pdb; pdb.set_trace()

model.py -> tf.enable_eager_execution()

In [5]:
%cd fail

/content/fail


In [6]:
pip install singleton_decorator

  Created wheel for singleton-decorator: filename=singleton_decorator-1.0.0-cp36-none-any.whl size=3125 sha256=187705c7c7967c99fab055c348c5296c55215e9d3376af5306fd3c9523b45a11
  Stored in directory: /root/.cache/pip/wheels/ce/43/87/9c4d65e727c32931aca54674ccf4f204e664306fbb507bcbd2
Successfully built singleton-decorator


In [7]:
import numpy as np
import os
import sys

from testbed import testbed
from jsma_experiments import JSMAUnconstrainedExperiment
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import model_eval, model_argmax, model_train
from dataset import Dataset
import tensorflow as tf

from cleverhans.loss import LossCrossEntropy
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import train, model_eval, model_argmax
from cleverhans.model_zoo.basic_cnn import ModelBasicCNN


def one_hot_encoder(ys, nb_classes):
    ohe_ys = []
    for y in ys:
        ohe_y = np.zeros(nb_classes)
        ohe_y[y] = 1

        ohe_ys.append(ohe_y)

    return np.array(ohe_ys)

class CNNModel:

    def __init__(self,scope):
        # Define input TF placeholder
        self.scope = scope
        self.reset()

    def reset(self):
        with tf.variable_scope(self.scope, reuse=tf.AUTO_REUSE):
            self.sess = tf.get_default_session()
            # Define input TF placeholder
            self.x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
            self.y = tf.placeholder(tf.float32, shape=(None, 10))
            import pdb; pdb.set_trace()
            self.model = ModelBasicCNN('model1', 10, 64)
            self.preds = self.model.get_logits(self.x)
            self.loss = LossCrossEntropy(self.model, smoothing=0.1)

    def train(self, dataset):
        train_params = {
            'nb_epochs': 1,
            'batch_size': 32,
            'learning_rate': 1e-2
        }
        with tf.variable_scope(self.scope,reuse=tf.AUTO_REUSE):
            self.sess.run(tf.global_variables_initializer())
            train(self.sess, self.loss, self.x, self.y, dataset.x, dataset.y, args=train_params)

    def test(self, dataset):
        with tf.variable_scope(self.scope, reuse=tf.AUTO_REUSE):
            eval_params = {'batch_size': 32}
            accuracy = model_eval(self.sess, self.x, self.y, self.preds, dataset.x, dataset.y, args=eval_params)
            print('Test accuracy on legitimate test examples: {0}'.format(accuracy))
            return {'accuracy': accuracy}

    def tf(self):
        return self.model


if __name__ == '__main__':
        #with testbed() as tb:
        tb = testbed()
        x_train, y_train, x_test, y_test = data_mnist()
        sess = tf.InteractiveSession()

        victim = CNNModel('victim')
        surrogate = CNNModel('surrogate')

        mnist = Dataset(np.append(x_train, x_test, axis=0), np.append(y_train, y_test, axis=0), train_pct=0.1)
        tb.register_dataset(mnist)
        tb.register_victim(victim)
        tb.register_surrogate(surrogate)

        e1 = JSMAUnconstrainedExperiment(['some_label'])
        # e2 = JSMALimitedFeatureKnowledgeExperiment(border=4)
        tb.register_experiment(e1)

        tb.run_experiments(runs=2, out_dir=os.path.join('output', 'jsma-FAIL'))


ModuleNotFoundError: ignored